In [1]:
import pandas as pd
import os

In [2]:
def get_noise_list(labels):
    """
    A function that generates a list of tuples that represents the start and end of noises.
    :param labels: the string that represents labels
    :return: a list of tuples that contain (start, end) of each noises.
    """
    starts = list()
    ends = list()
    
    for i in labels:
        starts.append(i['start'])
        ends.append(i['end'])
    
    noises = list()
    noises.append((0, starts[0])) # The start of file to starts[0] is considered a noise

    for i in range(len(ends)):
        try:
            noises.append((ends[i], starts[i + 1]))
        except IndexError:  # If index error, that means there is no more noise left
            pass

    noises.append((ends[len(ends) - 1], -1))  # The last end to the end of the file is considered a noise
    return noises

In [3]:
def generate_noise_df(df):
    """
    A function that generates a pandas.DataFrame that contains all start and ends of noises.
    :param df: a pandas.DataFrame object that represents the original dataframe
    :return: return a DataFrame object that includes all noise starts and ends.
    """
    file_names = list()
    start = list()
    end = list()
    class_name = list()
    
    for i in range(len(df)):
        cur_df = df.loc[i]
        noise_list = get_noise_list(eval(cur_df['labels']))
        cur_file_name = os.path.join(os.getcwd(), cur_df['audio'][1:])
        
        for noise in noise_list:
            start.append(noise[0])
            end.append(noise[1])
            class_name.append("noise")
            file_names.append(cur_file_name)

    modified_df = pd.DataFrame()
    modified_df['file_name'] = file_names
    modified_df['start'] = start
    modified_df['end'] = end
    modified_df['class'] = class_name
    
    return modified_df

In [4]:
def modify_df(file_name):
    """
    A function that reads file in csv format which was exported by LabelStudio.
    This function will transform given csv into dataframe that is more suitable for our sound classification.
    :param file_name: the string object that represents file name
    :returns: pandas.DataFrame object that represents data from the csv turned into our style.
    """
    data = pd.read_csv(file_name)
    
    file_names = list()
    start = list()
    end = list()
    class_name = list()

    for i in range(len(data)):
        instance = data.loc[i]
        cur_file_name = os.path.join(os.getcwd(), instance['audio'][1:])
        cur_class = eval(instance['intent'])  # Each values are recorded as str, so need eval
        cur_time_lines = eval(data.loc[i]['labels'])

        for j in range(len(cur_class)):
            cur_start = float(cur_time_lines[j]['start'])
            cur_end = float(cur_time_lines[j]['end'])

            start.append(cur_start)
            end.append(cur_end)
            class_name.append(cur_class[j])
            file_names.append(cur_file_name)

    modified_df = pd.DataFrame()
    modified_df['file_name'] = file_names
    modified_df['start'] = start
    modified_df['end'] = end
    modified_df['class'] = class_name
    
    noise_df = generate_noise_df(data)
    modified_df = modified_df.append(noise_df, ignore_index=True)
    
    return modified_df

In [5]:
from PIL import Image
from pydub import AudioSegment
import sqlite3
import string
import os

def generate_db():
    """
    A function that generates training_data/data.db file and initializes the table values.
    :return: None
    """
    conn = sqlite3.connect("./training_data/data.db")
    cur = conn.cursor()
    conn.execute("CREATE TABLE data(data_name TEXT, data_count INTEGER)")
    
    default_data = [(x, 0) for x in string.ascii_lowercase]
    default_data = default_data + [(str(x), 0) for x in range(0, 10, 1)]
    default_data.append(("noise", 0))
    
    cur.executemany("INSERT INTO data VALUES(?, ?)", default_data)
    
    conn.commit()
    conn.close()
    
def get_data_count(data_name):
    """
    A function that gets data count from data.db for specific data_name
    :param data_name: the data_name to look for data counts
    :return: returns count of data_count value for provided data_name, returns -1 if data_name does not exist.
    """
    conn = sqlite3.connect("./training_data/data.db")
    cur = conn.cursor()
    cur.execute("SELECT data_count FROM data WHERE data_name=\"" + data_name + "\"")
    rows = cur.fetchall()
    try:
        return rows[0][0]
    except IndexError:
        return -1
    
    conn.close()
    
def update_data_count(data_name):
    """
    A function that adds 1 to the current data_count for specified data_name
    :param data_name: the data_name to update values into
    :return: None
    """
    before_count = get_data_count(data_name)    
    new_count = before_count + 1
    
    conn = sqlite3.connect("./training_data/data.db")
    cur = conn.cursor()
    cur.execute("UPDATE data SET data_count=" + str(new_count) + " WHERE data_name=\"" + data_name + "\"")
    conn.commit()
    conn.close()

def generate_all_directories():
    """
    A function that generates training_data directories.
    This will generate trainig_data/a~z and 0~9 directories.
    :return: None
    """
    dir_names = [x for x in string.ascii_lowercase]
    dir_names = dir_names + [str(x) for x in range(0, 10, 1)]
    dir_names.append("noise")


    training_data_path = os.path.join(os.getcwd(), "training_data")

    if not os.path.exists(training_data_path):  # generate training_data directory
        os.mkdir(training_data_path)
    
    for i in dir_names:  # generate subdirectories such as a ~ z, 0 ~ 9
        new_dir= os.path.join(training_data_path, i)
        if not os.path.exists(new_dir):
            os.mkdir(new_dir)

def process_df(df):
    """
    A function that splits each sound classifications from dataframe into different files
    :param df: the pandas.DataFrame object that represents single row of information.
    """
    file_name = df['file_name']
    new_audio = AudioSegment.from_wav(file_name)
    
    start = df['start'] * 1000
    
    if df['end'] == -1:
        end = len(new_audio)
    else:
        end = df['end'] * 1000
    class_name = df['class']
    
    tmp_audio = new_audio[start:end]
    
    save_dir = os.path.join(os.getcwd(), "training_data", class_name)
    new_file_name = str(get_data_count(class_name)) + ".wav"
    new_file_name = os.path.join(save_dir, new_file_name)
    update_data_count(class_name)
    tmp_audio.export(new_file_name, format="wav")

In [6]:
df = modify_df("project-3-at-2022-06-23-07-16-c998cdd4.csv")

In [7]:
df

,file_name,start,end,class
0,/home/gooday2die/projects/Anti-Captcha-Sound/d...,0.018023,0.702913,5
1,/home/gooday2die/projects/Anti-Captcha-Sound/d...,1.111443,1.730247,t
2,/home/gooday2die/projects/Anti-Captcha-Sound/d...,1.850403,2.517269,j
3,/home/gooday2die/projects/Anti-Captcha-Sound/d...,2.865722,3.640728,t
4,/home/gooday2die/projects/Anti-Captcha-Sound/d...,3.893056,4.620000,n
...,...,...,...,...
2189,/home/gooday2die/projects/Anti-Captcha-Sound/d...,0.560398,1.251121,noise
2190,/home/gooday2die/projects/Anti-Captcha-Sound/d...,1.883198,2.515274,noise
2191,/home/gooday2die/projects/Anti-Captcha-Sound/d...,2.906250,3.440583,noise
2192,/home/gooday2die/projects/Anti-Captcha-Sound/d...,3.948850,4.378923,noise


In [8]:
df

,file_name,start,end,class
0,/home/gooday2die/projects/Anti-Captcha-Sound/d...,0.018023,0.702913,5
1,/home/gooday2die/projects/Anti-Captcha-Sound/d...,1.111443,1.730247,t
2,/home/gooday2die/projects/Anti-Captcha-Sound/d...,1.850403,2.517269,j
3,/home/gooday2die/projects/Anti-Captcha-Sound/d...,2.865722,3.640728,t
4,/home/gooday2die/projects/Anti-Captcha-Sound/d...,3.893056,4.620000,n
...,...,...,...,...
2189,/home/gooday2die/projects/Anti-Captcha-Sound/d...,0.560398,1.251121,noise
2190,/home/gooday2die/projects/Anti-Captcha-Sound/d...,1.883198,2.515274,noise
2191,/home/gooday2die/projects/Anti-Captcha-Sound/d...,2.906250,3.440583,noise
2192,/home/gooday2die/projects/Anti-Captcha-Sound/d...,3.948850,4.378923,noise


In [11]:
try:
    generate_db()
    print("[+] DB not found, generating a new one")
except sqlite3.OperationalError:  # when db already exists
    pass
generate_all_directories()

for i in range(len(df)):
    print("[+] Processing file " + str(i + 1) + " / " + str(len(df)))
    process_df(df.loc[i])

[+] DB not found, generating a new one
[+] Processing file 1 / 2194
[+] Processing file 2 / 2194
[+] Processing file 3 / 2194
[+] Processing file 4 / 2194
[+] Processing file 5 / 2194
[+] Processing file 6 / 2194
[+] Processing file 7 / 2194
[+] Processing file 8 / 2194
[+] Processing file 9 / 2194
[+] Processing file 10 / 2194
[+] Processing file 11 / 2194
[+] Processing file 12 / 2194
[+] Processing file 13 / 2194
[+] Processing file 14 / 2194
[+] Processing file 15 / 2194
[+] Processing file 16 / 2194
[+] Processing file 17 / 2194
[+] Processing file 18 / 2194
[+] Processing file 19 / 2194
[+] Processing file 20 / 2194
[+] Processing file 21 / 2194
[+] Processing file 22 / 2194
[+] Processing file 23 / 2194
[+] Processing file 24 / 2194
[+] Processing file 25 / 2194
[+] Processing file 26 / 2194
[+] Processing file 27 / 2194
[+] Processing file 28 / 2194
[+] Processing file 29 / 2194
[+] Processing file 30 / 2194
[+] Processing file 31 / 2194
[+] Processing file 32 / 2194
[+] Proces

[+] Processing file 292 / 2194
[+] Processing file 293 / 2194
[+] Processing file 294 / 2194
[+] Processing file 295 / 2194
[+] Processing file 296 / 2194
[+] Processing file 297 / 2194
[+] Processing file 298 / 2194
[+] Processing file 299 / 2194
[+] Processing file 300 / 2194
[+] Processing file 301 / 2194
[+] Processing file 302 / 2194
[+] Processing file 303 / 2194
[+] Processing file 304 / 2194
[+] Processing file 305 / 2194
[+] Processing file 306 / 2194
[+] Processing file 307 / 2194
[+] Processing file 308 / 2194
[+] Processing file 309 / 2194
[+] Processing file 310 / 2194
[+] Processing file 311 / 2194
[+] Processing file 312 / 2194
[+] Processing file 313 / 2194
[+] Processing file 314 / 2194
[+] Processing file 315 / 2194
[+] Processing file 316 / 2194
[+] Processing file 317 / 2194
[+] Processing file 318 / 2194
[+] Processing file 319 / 2194
[+] Processing file 320 / 2194
[+] Processing file 321 / 2194
[+] Processing file 322 / 2194
[+] Processing file 323 / 2194
[+] Proc

[+] Processing file 572 / 2194
[+] Processing file 573 / 2194
[+] Processing file 574 / 2194
[+] Processing file 575 / 2194
[+] Processing file 576 / 2194
[+] Processing file 577 / 2194
[+] Processing file 578 / 2194
[+] Processing file 579 / 2194
[+] Processing file 580 / 2194
[+] Processing file 581 / 2194
[+] Processing file 582 / 2194
[+] Processing file 583 / 2194
[+] Processing file 584 / 2194
[+] Processing file 585 / 2194
[+] Processing file 586 / 2194
[+] Processing file 587 / 2194
[+] Processing file 588 / 2194
[+] Processing file 589 / 2194
[+] Processing file 590 / 2194
[+] Processing file 591 / 2194
[+] Processing file 592 / 2194
[+] Processing file 593 / 2194
[+] Processing file 594 / 2194
[+] Processing file 595 / 2194
[+] Processing file 596 / 2194
[+] Processing file 597 / 2194
[+] Processing file 598 / 2194
[+] Processing file 599 / 2194
[+] Processing file 600 / 2194
[+] Processing file 601 / 2194
[+] Processing file 602 / 2194
[+] Processing file 603 / 2194
[+] Proc

[+] Processing file 863 / 2194
[+] Processing file 864 / 2194
[+] Processing file 865 / 2194
[+] Processing file 866 / 2194
[+] Processing file 867 / 2194
[+] Processing file 868 / 2194
[+] Processing file 869 / 2194
[+] Processing file 870 / 2194
[+] Processing file 871 / 2194
[+] Processing file 872 / 2194
[+] Processing file 873 / 2194
[+] Processing file 874 / 2194
[+] Processing file 875 / 2194
[+] Processing file 876 / 2194
[+] Processing file 877 / 2194
[+] Processing file 878 / 2194
[+] Processing file 879 / 2194
[+] Processing file 880 / 2194
[+] Processing file 881 / 2194
[+] Processing file 882 / 2194
[+] Processing file 883 / 2194
[+] Processing file 884 / 2194
[+] Processing file 885 / 2194
[+] Processing file 886 / 2194
[+] Processing file 887 / 2194
[+] Processing file 888 / 2194
[+] Processing file 889 / 2194
[+] Processing file 890 / 2194
[+] Processing file 891 / 2194
[+] Processing file 892 / 2194
[+] Processing file 893 / 2194
[+] Processing file 894 / 2194
[+] Proc

[+] Processing file 1147 / 2194
[+] Processing file 1148 / 2194
[+] Processing file 1149 / 2194
[+] Processing file 1150 / 2194
[+] Processing file 1151 / 2194
[+] Processing file 1152 / 2194
[+] Processing file 1153 / 2194
[+] Processing file 1154 / 2194
[+] Processing file 1155 / 2194
[+] Processing file 1156 / 2194
[+] Processing file 1157 / 2194
[+] Processing file 1158 / 2194
[+] Processing file 1159 / 2194
[+] Processing file 1160 / 2194
[+] Processing file 1161 / 2194
[+] Processing file 1162 / 2194
[+] Processing file 1163 / 2194
[+] Processing file 1164 / 2194
[+] Processing file 1165 / 2194
[+] Processing file 1166 / 2194
[+] Processing file 1167 / 2194
[+] Processing file 1168 / 2194
[+] Processing file 1169 / 2194
[+] Processing file 1170 / 2194
[+] Processing file 1171 / 2194
[+] Processing file 1172 / 2194
[+] Processing file 1173 / 2194
[+] Processing file 1174 / 2194
[+] Processing file 1175 / 2194
[+] Processing file 1176 / 2194
[+] Processing file 1177 / 2194
[+] Proc

[+] Processing file 1406 / 2194
[+] Processing file 1407 / 2194
[+] Processing file 1408 / 2194
[+] Processing file 1409 / 2194
[+] Processing file 1410 / 2194
[+] Processing file 1411 / 2194
[+] Processing file 1412 / 2194
[+] Processing file 1413 / 2194
[+] Processing file 1414 / 2194
[+] Processing file 1415 / 2194
[+] Processing file 1416 / 2194
[+] Processing file 1417 / 2194
[+] Processing file 1418 / 2194
[+] Processing file 1419 / 2194
[+] Processing file 1420 / 2194
[+] Processing file 1421 / 2194
[+] Processing file 1422 / 2194
[+] Processing file 1423 / 2194
[+] Processing file 1424 / 2194
[+] Processing file 1425 / 2194
[+] Processing file 1426 / 2194
[+] Processing file 1427 / 2194
[+] Processing file 1428 / 2194
[+] Processing file 1429 / 2194
[+] Processing file 1430 / 2194
[+] Processing file 1431 / 2194
[+] Processing file 1432 / 2194
[+] Processing file 1433 / 2194
[+] Processing file 1434 / 2194
[+] Processing file 1435 / 2194
[+] Processing file 1436 / 2194
[+] Proc

[+] Processing file 1694 / 2194
[+] Processing file 1695 / 2194
[+] Processing file 1696 / 2194
[+] Processing file 1697 / 2194
[+] Processing file 1698 / 2194
[+] Processing file 1699 / 2194
[+] Processing file 1700 / 2194
[+] Processing file 1701 / 2194
[+] Processing file 1702 / 2194
[+] Processing file 1703 / 2194
[+] Processing file 1704 / 2194
[+] Processing file 1705 / 2194
[+] Processing file 1706 / 2194
[+] Processing file 1707 / 2194
[+] Processing file 1708 / 2194
[+] Processing file 1709 / 2194
[+] Processing file 1710 / 2194
[+] Processing file 1711 / 2194
[+] Processing file 1712 / 2194
[+] Processing file 1713 / 2194
[+] Processing file 1714 / 2194
[+] Processing file 1715 / 2194
[+] Processing file 1716 / 2194
[+] Processing file 1717 / 2194
[+] Processing file 1718 / 2194
[+] Processing file 1719 / 2194
[+] Processing file 1720 / 2194
[+] Processing file 1721 / 2194
[+] Processing file 1722 / 2194
[+] Processing file 1723 / 2194
[+] Processing file 1724 / 2194
[+] Proc

[+] Processing file 1981 / 2194
[+] Processing file 1982 / 2194
[+] Processing file 1983 / 2194
[+] Processing file 1984 / 2194
[+] Processing file 1985 / 2194
[+] Processing file 1986 / 2194
[+] Processing file 1987 / 2194
[+] Processing file 1988 / 2194
[+] Processing file 1989 / 2194
[+] Processing file 1990 / 2194
[+] Processing file 1991 / 2194
[+] Processing file 1992 / 2194
[+] Processing file 1993 / 2194
[+] Processing file 1994 / 2194
[+] Processing file 1995 / 2194
[+] Processing file 1996 / 2194
[+] Processing file 1997 / 2194
[+] Processing file 1998 / 2194
[+] Processing file 1999 / 2194
[+] Processing file 2000 / 2194
[+] Processing file 2001 / 2194
[+] Processing file 2002 / 2194
[+] Processing file 2003 / 2194
[+] Processing file 2004 / 2194
[+] Processing file 2005 / 2194
[+] Processing file 2006 / 2194
[+] Processing file 2007 / 2194
[+] Processing file 2008 / 2194
[+] Processing file 2009 / 2194
[+] Processing file 2010 / 2194
[+] Processing file 2011 / 2194
[+] Proc

In [10]:
from IPython.display import Audio 
#Audio("/home/gooday2die/projects/Anti-Captcha/sounddooo/sounds/training_data/j/0.wav", autoplay=True)